# Heat System with Storage

District heating with a thermal storage tank and time-varying gas prices.

This notebook introduces:

- **Storage**: Thermal buffer with charge/discharge flows, efficiency, and self-discharge
- **Time-varying prices**: Cost coefficients that change per timestep
- **Load shifting**: The optimizer charges storage when gas is cheap and discharges when expensive

## Setup

In [ ]:
import math
from datetime import datetime, timedelta

import plotly.express as px
import plotly.io as pio

from fluxopt import Bus, Converter, Effect, Flow, Port, Storage, optimize

pio.renderers.default = 'notebook_connected'

## Generate Time Series Data

We model 48 hours with hourly resolution. Heat demand follows a day/night
pattern, and gas prices have off-peak/peak tariffs.

In [ ]:
n_hours = 48
timesteps = [datetime(2024, 1, 15) + timedelta(hours=h) for h in range(n_hours)]

# Heat demand: sinusoidal pattern - peaks during business hours, low at night
heat_demand = [50 + 40 * max(0.0, math.sin(math.pi * ((h % 24) - 6) / 12)) for h in range(n_hours)]

# Gas price: off-peak (22:00-06:00) at 0.04 EUR/kWh, peak at 0.08 EUR/kWh
gas_price = [0.04 if (h % 24) < 6 or (h % 24) >= 22 else 0.08 for h in range(n_hours)]

Visualize the demand and price profiles:

In [ ]:
px.bar(x=timesteps, y=heat_demand, labels={'x': 'Time', 'y': 'kW'}, title='Heat Demand')

In [ ]:
px.line(x=timesteps, y=gas_price, labels={'x': 'Time', 'y': 'EUR/kWh'}, title='Gas Price')

## Build and Solve the Energy System

The system:

```
Gas Grid ──► [gas] ──► Boiler ──► [heat] ◄──► Storage
                         η=92%       │
                                     ▼
                                   Office
```

In [ ]:
# Gas supply with time-varying price
gas_source = Flow(bus='gas', size=500, effects_per_flow_hour={'cost': gas_price})

# Boiler: 150 kW thermal capacity, 92% efficiency
fuel = Flow(bus='gas', size=300)
heat_out = Flow(bus='heat', size=150)

# Demand: normalize to relative profile
max_demand = max(heat_demand)
demand_profile = [d / max_demand for d in heat_demand]
demand_flow = Flow(bus='heat', size=max_demand, fixed_relative_profile=demand_profile)

# Thermal storage: 500 kWh, 98% charge/discharge efficiency, 0.5%/h self-discharge
charge_flow = Flow(bus='heat', size=100)
discharge_flow = Flow(bus='heat', size=100)
storage = Storage(
    'thermal_storage',
    charging=charge_flow,
    discharging=discharge_flow,
    capacity=500,
    eta_charge=0.98,
    eta_discharge=0.98,
    relative_loss_per_hour=0.005,
    prior_level=250.0,  # start half-full (absolute MWh)
    cyclic=False,
)

result = optimize(
    timesteps=timesteps,
    buses=[Bus('gas'), Bus('heat')],
    effects=[Effect('cost', unit='EUR', is_objective=True)],
    ports=[
        Port('gas_grid', imports=[gas_source]),
        Port('office', exports=[demand_flow]),
    ],
    converters=[
        Converter.boiler('boiler', thermal_efficiency=0.92, fuel_flow=fuel, thermal_flow=heat_out),
    ],
    storages=[storage],
)

## Analyze Results

### Cost Summary

In [ ]:
total_heat = sum(heat_demand)

print(f'Total heat delivered: {total_heat:.0f} kWh')
print(f'Total cost:          {result.objective:.2f} EUR')
print(f'Avg cost of heat:    {result.objective / total_heat * 100:.2f} ct/kWh')

### Flow Rates

All flow rates across the horizon:

In [ ]:
df = result.flow_rates.to_dataframe('kW').reset_index()
px.line(df, x='time', y='kW', color='flow', title='Flow Rates', line_shape='hv')

### Storage Operation

Charge and discharge rates show the load-shifting behavior:

In [ ]:
storage_flows = [f for f in result.flow_rates.coords['flow'].values if 'thermal_storage' in f]
df = result.flow_rates.sel(flow=storage_flows).to_dataframe('kW').reset_index()
px.line(df, x='time', y='kW', color='flow', title='Storage Charge / Discharge', line_shape='hv')

### Storage Level

The stored energy over time:

In [ ]:
df = result.storage_level('thermal_storage').to_dataframe('kWh').reset_index()
px.area(df, x='time', y='kWh', title='Storage Level', line_shape='hv')

### Heat Balance

How boiler output and storage work together to meet demand:

In [ ]:
heat_flow_ids = [
    f for f in result.flow_rates.coords['flow'].values if any(k in f for k in ('heat', 'charge', 'discharge'))
]
df = result.flow_rates.sel(flow=heat_flow_ids).to_dataframe('kW').reset_index()
px.line(df, x='time', y='kW', color='flow', title='Heat Bus Balance', line_shape='hv')

### Effect Totals

In [ ]:
df = result.effects_temporal.to_dataframe('EUR').reset_index()
px.bar(df, x='time', y='EUR', color='effect', title='Effects per Timestep')

## Key Insights

The optimization reveals how storage enables **load shifting**:

1. **Charge during off-peak**: When gas is cheap (night), the boiler runs at
   higher output to charge the storage
2. **Discharge during peak**: During expensive periods, storage supplements the
   boiler, reducing gas purchases
3. **Efficiency trade-off**: Storage has round-trip losses (0.98 x 0.98 = 96%)
   plus self-discharge, so shifting only happens when the price spread exceeds
   the loss cost

## Summary

You learned how to:

- Add **Storage** with capacity, efficiency, and self-discharge
- Use **time-varying prices** via `effects_per_flow_hour`
- Inspect **storage levels** and **storage operation**
- Use xarray DataArrays for result analysis